# EDA (PM10, PM2,5)

*alpha 1.0*

* PM10, PM2,5, O3, NO2, SO2, C6H6, CO - data from GIOŚ archive: https://powietrze.gios.gov.pl/pjp/archives
* Prediction will be based on weather (last 24h and current - forcast for predicting future pollutant values) - data from IMGW archive: https://dane.imgw.pl/data/dane_pomiarowo_obserwacyjne/dane_meteorologiczne/terminowe/synop/
* weather forcast / current params - https://openweathermap.org/api
* All pollutants are time series, 1 measure per hour
* time period used: last 4 years (2016, 2017, 2018, 2019)

## env

In [12]:
import os
import sys


import pandas as pd
pd.options.display.max_rows = None
pd.options.display.max_columns = None
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [23]:
pd.read_excel('../data/gios/2019_PM10_1g.xlsx').head(10)

,Nr,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139
0,Kod stacji,DsDusznikMOB,DsDzialoszyn,DsDziePilsud,DsJaworMOB,DsJelGorOgin,DsKlodzSzkol,DsLegAlRzecz,DsLubanMieszMOB,DsNowRudJezi,DsOlawZolnAK,DsWalbrzWyso,DsWrocWybCon,DsZabkPowWar,KpBydPlPozna,KpBydWarszaw,KpInowSolank,KpMogiNowMOB,KpToruDziewu,KpToruKaszow,KpToruWSikor,KpWloclKalis,KpWloclOkrze,LbBiaPodOrze,LbChelPolan,LbLubObywate,LbNaleczow,LbZamoHrubie,LdGajewUjWod,LdLodzCzerni,LdLodzGdansk,LdLodzJanPaw,LdPabiKonsta,LdPioTrKraPr,LdRadomsRoln,LdZgieMielcz,LuGorzKosGdy,LuNowaSolMOB,LuSulecDudka,LuWsKaziWiel,LuZarySzyman,LuZielKrotka,MpKrakAlKras,MpKrakBujaka,MpKrakBulwar,MpKrakDietla,MpKrakOsPias,MpKrakSwoszo,MpKrakWadow,MpKrakZloRog,MpNoSaczNadb,MpNoTargPSlo,MpSkawOsOgro,MpSuchaNiesz,MpTarBitStud,MpTarRoSitko,MpTrzebOsZWM,MpZakopaSien,MzKonJezMos,MzLegZegrzyn,MzMinMazKaziMOB,MzOtwoBrzozo,MzPiasPulask,MzPlocMiReja,MzRadTochter,MzSiedKonars,MzWarAlNiepo,MzWarBajkowa,MzWarChrosci,MzWarKondrat,MzWarTolstoj,MzWarWokalna,MzZyraRoosev,OpOlesSlowac,OpOpoleKoszy,OpPrudPodgor,OpZdziePiast,PdBialWarsza,PdBorsukowiz,PdLomSikorsk,PkDebiGrottg,PkHorZdrParkMOB,PkJarosPruch,PkJasloSikor,PkMielBierna,PkNiskoSzkla,PkPrzemGrunw,PkRymZdrPark,PkRzeszPilsu,PkRzeszRejta,PmGdaKacze02,PmGdaLeczk08,PmGdaPoWie01,PmGdaWyzwo03,PmGdyPoreb04,PmGdySzaf09N,PmKosTargo12,PmLebMalcz16,PmMalMicki15,PmSlupKniazi,PmSopBitPl06,SkKielTargow,SkMaloSlonec,SkNowiParkow,SkPolaRuszcz,SkSkarZielnaMOB,SkStaraZlota,SlBielKossak,SlCiesMickie,SlCzestoArmK,SlCzestoBacz,SlDabro1000L,SlGliwicMewy,SlKatoKossut,SlLublSzymal,SlRybniBorki,SlSosnoLubel,SlTychyTolst,SlUstronSana,SlWodzGalczy,SlZabSkloCur,SlZawGalczyn,SlZlotPotLes,SlZywieKoper,WmElbBazynsk,WmElkStadion,WmGoldJacwie,WmGoldUzdrowMOB,WmOlsPuszkin,WmOstrPilsud,WpBoroDrapal,WpKoniWyszyn,WpKozieosLes,WpPoznDabrow,WpPoznRatajeMOB,ZpKoszArKraj,ZpSzczAndr01,ZpSzczBudzWosMOB,ZpSzczecPrze,ZpSzczPils02
1,Wskaźnik,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10,PM10
2,Czas uśredniania,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g,1g
3,Jednostka,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,ug/m3,u

In [24]:
pm10_temp = pd.read_excel('../data/gios/2019_PM10_1g.xlsx', skiprows=5)

In [25]:
pm10_temp.head(10)

,Kod stanowiska,DsDusznikMOB-PM10-1g,DsDzialoszyn-PM10-1g,DsDziePilsud-PM10-1g,DsJaworMOB-PM10-1g,DsJelGorOgin-PM10-1g,DsKlodzSzkol-PM10-1g,DsLegAlRzecz-PM10-1g,DsLubanMieszMOB-PM10-1g,DsNowRudJezi-PM10-1g,DsOlawZolnAK-PM10-1g,DsWalbrzWyso-PM10-1g,DsWrocWybCon-PM10-1g,DsZabkPowWar-PM10-1g,KpBydPlPozna-PM10-1g,KpBydWarszaw-PM10-1g,KpInowSolank-PM10-1g,KpMogiNowMOB-PM10-1g,KpToruDziewu-PM10-1g,KpToruKaszow-PM10-1g,KpToruWSikor-PM10-1g,KpWloclKalis-PM10-1g,KpWloclOkrze-PM10-1g,LbBiaPodOrze-PM10-1g,LbChelPolan-PM10-1g,LbLubObywate-PM10-1g,LbNaleczow-PM10-1g,LbZamoHrubie-PM10-1g,LdGajewUjWod-PM10-1g,LdLodzCzerni-PM10-1g,LdLodzGdansk-PM10-1g,LdLodzJanPaw-PM10-1g,LdPabiKonsta-PM10-1g,LdPioTrKraPr-PM10-1g,LdRadomsRoln-PM10-1g,LdZgieMielcz-PM10-1g,LuGorzKosGdy-PM10-1g,LuNowaSolMOB-PM10-1g,LuSulecDudka-PM10-1g,LuWsKaziWiel-PM10-1g,LuZarySzyman-PM10-1g,LuZielKrotka-PM10-1g,MpKrakAlKras-PM10-1g,MpKrakBujaka-PM10-1g,MpKrakBulwar-PM10-1g,MpKrakDietla-PM10-1g,MpKrakOsPias-PM10-1g,MpKrakSwoszo-PM10-1g,MpKrakWadow-PM10-1g,MpKrakZloRog-PM10-1g,MpNoSaczNadb-PM10-1g,MpNoTargPSlo-PM10-1g,MpSkawOsOgro-PM10-1g,MpSuchaNiesz-PM10-1g,MpTarBitStud-PM10-1g,MpTarRoSitko-PM10-1g,MpTrzebOsZWM-PM10-1g,MpZakopaSien-PM10-1g,MzKonJezMos-PM10-1g,MzLegZegrzyn-PM10-1g,MzMinMazKaziMOB-PM10-1g,MzOtwoBrzozo-PM10-1g,MzPiasPulask-PM10-1g,MzPlocMiReja-PM10-1g,MzRadTochter-PM10-1g,MzSiedKonars-PM10-1g,MzWarAlNiepo-PM10-1g,MzWarBajkowa-PM10-1g,MzWarChrosci-PM10-1g,MzWarKondrat-PM10-1g,MzWarTolstoj-PM10-1g,MzWarWokalna-PM10-1g,MzZyraRoosev-PM10-1g,OpOlesSlowac-PM10-1g,OpOpoleKoszy-PM10-1g,OpPrudPodgor-PM10-1g,OpZdziePiast-PM10-1g,PdBialWarsza-PM10-1g,PdBorsukowiz-PM10-1g,PdLomSikorsk-PM10-1g,PkDebiGrottg-PM10-1g,PkHorZdrParkMOB-PM10-1g,PkJarosPruch-PM10-1g,PkJasloSikor-PM10-1g,PkMielBierna-PM10-1g,PkNiskoSzkla-PM10-1g,PkPrzemGrunw-PM10-1g,PkRymZdrPark-PM10-1g,PkRzeszPilsu-PM10-1g,PkRzeszRejta-PM10-1g,PmGdaKacze02-PM10-1g,PmGdaLeczk08-PM10-1g,PmGdaPoWie01-PM10-1g,PmGdaWyzwo03-PM10-1g,PmGdyPoreb04-PM10-1g,PmGdySzaf09N-PM10-1g,PmKosTargo12-PM10-1g,PmLebMalcz16-PM10-1g,PmMalMicki15-PM10-1g,PmSlupKniazi-PM10-1g,PmSopBitPl06-PM10-1g,SkKielTargow-PM10-1g,SkMaloSlonec-PM10-1g,SkNowiParkow-PM10-1g,SkPolaRuszcz-PM10-1g,SkSkarZielnaMOB-PM10-1g,SkStaraZlota-PM10-1g,SlBielKossak-PM10-1g,SlCiesMickie-PM10-1g,SlCzestoArmK-PM10-1g,SlCzestoBacz-PM10-1g,SlDabro1000L-PM10-1g,SlGliwicMewy-PM10-1g,SlKatoKossut-PM10-1g,SlLublSzymal-PM10-1g,SlRybniBorki-PM10-1g,SlSosnoLubel-PM10-1g,SlTychyTolst-PM10-1g,SlUstronSana-PM10-1g,SlWodzGalczy-PM10-1g,SlZabSkloCur-PM10-1g,SlZawGalczyn-PM10-1g,SlZlotPotLes-PM10-1g,SlZywieKoper-PM10-1g,WmElbBazynsk-PM10-1g,WmElkStadion-PM10-1g,WmGoldJacwie-PM10-1g,WmGoldUzdrowMOB-PM10-1g,WmOlsPuszkin-PM10-1g,WmOstrPilsud-PM10-1g,WpBoroDrapal-PM10-1g,WpKoniWyszyn-PM10-1g,WpKozieosLes-PM10-1g,WpPoznDabrow-PM10-1g,WpPoznRatajeMOB-PM10-1g,ZpKoszArKraj-PM10-1g,ZpSzczAndr01-PM10-1g,ZpSzczBudzWosMOB-PM10-1g,ZpSzczecPrze-PM10-1g,ZpSzczPils02-PM10-1g
0,2019-01-01 01:00:00,35.21080,16.68570,245.92900,55.28140,125.32600,306.4300,65.3853,31.1785,NaN,39.710,35.29540,115.5170,46.40820,94.5,114.2649,15.1050,NaN,86.496,NaN,26.7,NaN,53.753,NaN,14.0,65.3200,NaN,99.7014,38.0,70.0,162.294,35.0,53.0,42.0,37.0,69.040,57.03110,76.34730,17.4035,58.2072,152.5210,36.9684,88.2185,139.7900,120.0570,73.4495,161.0530,NaN,83.6841,77.8355,63.7522,132.9710,NaN,72.2746,67.43160,73.1073,113.0970,138.17600,39.357284,48.155374,NaN,39.772007,NaN,38.098950,40.389135,69.48,73.077490,59.870348,62.6603,54.113472,NaN,46.527342,48.724218,24.2266,38.08460,NaN,50.4629,43.8,8.5,76.0,45.1,NaN,36.6,23.4,69.4362,36.4,50.4,19.3,35.9147,31.1,23.78530,25.73020,76.30860,16.39110,34.67610,22.19170,56.37780,69.17820,25.67840,NaN,24.03820,50.0,31.72,25.80,18.78,NaN,32.69112,169.5440,62.6338,83.2158,58.4966,119.2360,130.7090,84.9443,60.1095,63.3477,55.9938,54.9406,20.58670,175.2180,151.9340,NaN,26.6439,67.76040,34.21110,25.1411,18.62170,15.77440,44.76810,36.03420,26.20300,34.6517,30.17020,82.75970,NaN,17.41420,NaN,NaN,50.28900,7

In [26]:
pm10_temp.shape

(8760, 140)

In [27]:
24*365

8760

In [ ]:
pm10 = []

for year in [2016, 2017, 2018, 2019]:
    